# 1. Analysis of Given Features

We analyzed the correlation level of features from given data ('tr_eikon_eod_data.csv').

Conclusion: Upon examining the provided data ('tr_eikon_eod_data.csv'), we found weak correlations (magnitude < 0.04) between the given features and stock direction. To improve prediction accuracy, we'll need to incorporate additional or newly created features.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./data/tr_eikon_eod_data.csv', index_col = 0, parse_dates = True)
data = data[['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N', 'SPY']]
data.dropna(inplace = True)

# Direction: 1 means that tomorrow's price is >= today's price and -1 means that tomorrow's price < today's price
data['direction'] = np.where(data['AAPL.O'] <= data['AAPL.O'].shift(-1), 1, -1)

data

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,direction
Date,,,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08,113.33,1
2010-01-05,30.625684,30.960,20.87,134.69,176.14,113.63,-1
2010-01-06,30.138541,30.770,20.80,132.25,174.26,113.71,-1
2010-01-07,30.082827,30.452,20.60,130.00,177.67,114.19,1
2010-01-08,30.282827,30.660,20.83,133.52,174.31,114.57,-1
...,...,...,...,...,...,...,...
2018-06-25,182.170000,98.390,50.71,1663.15,221.54,271.00,1
2018-06-26,184.430000,99.080,49.67,1691.09,221.58,271.60,-1
2018-06-27,184.160000,97.540,48.76,1660.51,220.18,269.35,1


In [3]:
# Correlation Analysis for each feature
data.corr()['direction']

AAPL.O      -0.036055
MSFT.O      -0.016139
INTC.O      -0.021609
AMZN.O      -0.021744
GS.N        -0.009771
SPY         -0.021940
direction    1.000000
Name: direction, dtype: float64

# 2. Additional Features: Feature Collection from Online Data

In [4]:
import requests
import pandas as pd

In [5]:
API_KEY_ALPHA_VANTAGE = "ICAG0XBGXUELT5PD"

In [9]:
def add_features(input_file, stock_symbol, api_urls, api_paths):
    df = pd.read_csv(input_file, index_col=0, parse_dates=True)
    df = df[stock_symbol]
    for (url, paths) in zip(api_urls, api_paths):
        r = requests.get(url)
        data = r.json()
        print(data)
        for path in paths:
            data = data[path]
        new_df = pd.DataFrame(data).transpose()
        new_df.index = pd.to_datetime(new_df.index)
        df = pd.merge(df, new_df, left_index=True, right_index=True, how='left')
    return df

input_file = './data/microsoft_output.csv'
output_file = './data/microsoft_output.csv'
stock_name = 'MSFT'
stock_symbol = 'MSFT.O'
api_urls = [
    f'https://www.alphavantage.co/query?function=STOCH&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=RSI&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=ADX&symbol={stock_name}&interval=daily&time_period=10&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=CCI&symbol={stock_name}&interval=daily&time_period=10&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=AROON&symbol={stock_name}&interval=daily&time_period=14&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=OBV&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=AD&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=SMA&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=EMA&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}'
    ]
api_paths = [
    ['Technical Analysis: STOCH'],
    ['Technical Analysis: RSI'],
    ['Technical Analysis: ADX'],
    ['Technical Analysis: CCI'],
    ['Technical Analysis: AROON'],
    ['Technical Analysis: OBV'],
    ['Technical Analysis: Chaikin A/D'],
    ['Technical Analysis: SMA'],
    ['Technical Analysis: EMA']
    ]

df = add_features(input_file, stock_symbol, api_urls, api_paths)
df

{'Meta Data': {'1: Symbol': 'MSFT', '2: Indicator': 'Stochastic (STOCH)', '3: Last Refreshed': '2024-04-09', '4: Interval': 'daily', '5.1: FastK Period': 5, '5.2: SlowK Period': 3, '5.3: SlowK MA Type': 0, '5.4: SlowD Period': 3, '5.5: SlowD MA Type': 0, '6: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: STOCH': {'2024-04-09': {'SlowK': '71.1111', 'SlowD': '50.1528'}, '2024-04-08': {'SlowK': '45.8859', 'SlowD': '33.6251'}, '2024-04-05': {'SlowK': '33.4615', 'SlowD': '32.1525'}, '2024-04-04': {'SlowK': '21.5279', 'SlowD': '33.6868'}, '2024-04-03': {'SlowK': '41.4679', 'SlowD': '37.4957'}, '2024-04-02': {'SlowK': '38.0646', 'SlowD': '28.7837'}, '2024-04-01': {'SlowK': '32.9547', 'SlowD': '24.7741'}, '2024-03-28': {'SlowK': '15.3319', 'SlowD': '29.9457'}, '2024-03-27': {'SlowK': '26.0356', 'SlowD': '50.1265'}, '2024-03-26': {'SlowK': '48.4697', 'SlowD': '70.6168'}, '2024-03-25': {'SlowK': '75.8743', 'SlowD': '80.6345'}, '2024-03-22': {'SlowK': '87.5064', 'SlowD': '76.8633'}, '2024-0

,MSFT.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA
Date,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,30.950,22.6012,20.0644,47.6624,36.1135,-200.4061,100.0000,64.2857,-170032596.0000,-7207872883.3625,22.5763,22.4974
2010-05-01,30.960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-01,30.770,58.4734,43.3955,22.6915,60.6866,-57.2855,78.5714,14.2857,-728850596.0000,-7440150402.1878,20.4367,20.3482
2010-07-01,30.452,9.2413,7.7560,20.6368,37.6466,-110.9580,100.0000,42.8571,-1307756396.0000,-7786346725.2856,19.1478,18.8035
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,98.390,36.1002,51.6045,47.7111,18.6535,-211.9001,100.0000,14.2857,-744599609.0000,-3823873542.8391,94.8803,94.6967
2018-06-26,99.080,28.9721,40.1854,41.0082,20.4706,-112.7838,92.8571,7.1429,-717702365.0000,-3837423282.2978,94.6660,94.3661
2018-06-27,97.540,19.8773,28.3165,46.3974,23.2159,-141.2104,85.7143,0.0000,-749000751.0000,-3865376802.6184,94.4649,94.2254


In [40]:
df.to_csv(output_file, index=True)

# 3. Additional Features: Feature Creation

In [25]:
SMA_short = 7
SMA_long = 20

def create_features(input_file, stock_name, output_name):
    df = pd.read_csv(input_file, index_col=0, parse_dates=True).dropna()   

    #create SMA_EMA_ratio, SMA_Short, SMA_Long and SMA_Long_Short feature
    df['SMA_EMA_ratio'] = df['SMA'] / df['EMA']
    df['SMA_Short'] = df[stock_name].rolling(SMA_short).mean()
    df['SMA_Long'] = df[stock_name].rolling(SMA_long).mean()
    df['SMA_Long_Short'] = df['SMA_Short'] / df['SMA_Long']
    
    df.to_csv(output_name, index=True)

file_name = './data/apple.csv'
stock_name = 'AAPL.O'
output_file = './data/apple.csv'

create_features(file_name, stock_name, output_file)